SwapAI is an AI-powered agent that helps users perform token swaps and transfers on the Base blockchain. It takes natural language requests like *“Swap 0.5 ETH to USDC with 1% slippage”* or *“Send 20 USDC to 0xabc…”*, extracts the parameters, and executes the right action using integrated tools. The agent can also handle general user questions beyond swaps and sends.


#### **The goal**

* Make blockchain interactions easier through conversational AI.
* Ensure clarity by confirming parameters before execution.
* Support structured outputs (markdown) for front-end rendering.
* Balance between tool usage (for swaps/sends) and general reasoning (for normal questions).

---

#### **Stack**

* **Python + Jupyter Notebook** (development & testing)
* **LangChain** (LLM orchestration, tools binding)
* **LangGraph** (state management, branching logic, tool handling)
* **Google Generative AI (Gemini)** or **OpenAI GPT** (LLM backbone)
* **dotenv** (manage API keys and environment variables)

---

#### **Why LangChain**

* Provides a clean interface to connect LLMs with tools.
* Offers message-based state management through LangGraph.
* Makes it easier to maintain context windows and structured workflows.
* Prebuilt utilities like `ToolNode` and `add_messages` reduce boilerplate.
* Extensible: easy to add new tools, memory, or different LLM backends.


---

#### Imports

1. ***from system_prompt import DEFAULT_SYSTEM_PROMPT***  
This imports the DEFAULT_SYSTEM_PROMPT from the system_prompt.py file in the repo.

2. ***from dotenv import load_dotenv***  
Loads environment variables from a .env file into your Python environment. This lets us keep API keys outside the code in a .env file with *GOOGLE_API_KEY=your API key* as its content

3. ***from typing import Annotated, Sequence, List, Optional***    
The typing module is used for type hunts.  

    **Annotated** adds metadata to a type.  

    **Sequence** means any ordered collection.    

    **List** is specifically a python list.  

    **Optional** means the value can be None.   

4. ***from typing_extensions import TypedDict***  
    **TypedDict** lets you define dictionary “schemas” with fixed keys and types.

5. ***from langgraph.graph import StateGraph, END***  

    **StateGraph** is the main component for building a state machine or a flowchart for an AI application.

    **END** is a special, built-in node that signifies the end of a process.

    StateGraph is used to orchestrate complex, multi-step workflows for an AI agent, and END is how you tell the workflow to stop.

6. ***from langchain_core.messages import AIMessage, HumanMessage, SystemMessage***  
These are data structures that represent the different roles in a conversation. They provide the structured format that chat models require to understand the flow and context of a conversation.  

    **BaseMessage:** The base class for all chat messages in LangChain.  

    *Subclasses:*  

        **HumanMessage:** Represents input from the end-user. 

        **AIMessage:** Represents a response generated by the AI model.

        **SystemMessage:** Sets the overall behavior, persona, or high-level instructions for the AI. 

        **ToolMessage:** Stores a tool's response. Needed so the conversation history includes not just what the AI says, but also what tools returned.

7. ***from langchain_google_genai import ChatGoogleGenerativeAI***  
This is the LangChain wrapper for Google's Gemini family of models. It allows the code to commmunicate with the Gemini API in a standardised way that fits into the LangChain ecosystem.

8. ***from langchain_core.tools import tool***  
This turns a python functioon into a tool that the AI can use. 

9. ***from langgraph.graph.message import add_messages***  
    **add_messages** is a reducer function. This means: every time a node returns new messages, instead of replacing the old ones, it appends them to the state automatically.

10. ***from langgraph.prebuilt import ToolNode***  
    **ToolNode** is a prebuilt node that knows how to call tools. You pass it your tools list, and it manages execution + returning results.

In [2]:
from system_prompt import DEFAULT_SYSTEM_PROMPT

from dotenv import load_dotenv 

from typing import Annotated, Sequence, List, Optional
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

In [ ]:
load_dotenv()   # This loads the .env file which contains the API key

True

#### Config Dictionary  
Defines system prompt, creativity (temperature), response length, and model.  

Keeps settings in one place → easier to tweak, cleaner code, and easier to switch models.

In [4]:
config = {
    "system_prompt": DEFAULT_SYSTEM_PROMPT,
    "temperature": 0.9,
    "max_output_tokens": 1008,
    "model": "gemini-2.5-flash-lite"
}

#### Class Creation

**AgentState**  

    Defines what data flows between nodes in your LangGraph (e.g., messages, tool results).

    Keeps the workflow consistent and prevents errors (e.g., wrong data types).

    Makes the agent easier to extend later (e.g., add memory, logs).  
    

**InstructionStep**

    Gives a strict schema for step-by-step instructions.

    Ensures output is structured and predictable for your frontend.

    Makes parsing and rendering simple (no guesswork in how the AI formats steps).

In [5]:
class AgentState(TypedDict):
    """Represents the state of the agent's workflow."""
    # A list of messages, annotated with a reducer to append new messages
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [ ]:
# Define a type for structured step-by-step instructions
class InstructionStep(TypedDict):
    # Short heading for the step
    title: str
    # List of detail lines (each step can have multiple sub-points)
    details: List[str]

#### Tool Definition

SwapAI uses three tools to handle different types of user intents:

##### `swap_tokens`

* **Purpose:** Handles token swaps on the Base network.
* **Example:** `Swap 0.5 ETH to USDC`.
* **Parameters:**

  * `amount` *(float)* → amount of tokens to swap.
  * `from_token` *(str)* → token being swapped from.
  * `to_token` *(str)* → token being swapped to.
  * `slippage` *(float, default: 1.0)* → allowed price movement during the swap.

---

##### `send_tokens`

* **Purpose:** Transfers tokens to another wallet address.
* **Example:** `Send 5 USDC to 0xabc...`.
* **Parameters:**

  * `amount` *(float)* → amount of tokens to send.
  * `token` *(str)* → token being transferred.
  * `recipient` *(str)* → wallet address of the recipient.

---

##### `give_instructions`

* **Purpose:** Provides step-by-step guidance in Markdown format.
* **Example:** `How do I connect my wallet?`.
* **Parameters:**

  * `topic` *(str)* → the subject the user needs guidance on.

---

This structure ensures SwapAI can both execute transactions and educate users depending on their request.



In [7]:
# Tools

@tool
def swap_tokens(amount: float, from_token: str, to_token: str, slippage: float = 1.0):
    """Produce a mock swap of tokens following the instructions in 'DEFAULT_SYSTEM_PROMPT'"""
    return {
        "status": "success",
        "tx_hash": "0xmockswap123",
        "details": f"Swapped {amount} {from_token} to {to_token} with {slippage}% slippage"
    }

# Mock tool to simulate sending tokens
@tool
def send_tokens(amount: float, token: str, recipient: str):
    """Produce a mock send transaction following the instructions in 'DEFAULT_SYSTEM_PROMPT '"""
    return {
        "status": "success",
        "tx_hash": "0xmocksend456",
        "details": f"Sent {amount} {token} to {recipient}"
    }

@tool
def give_instructions(
    title: str,
    summary: str,
    steps: List[InstructionStep],
    notes: Optional[List[str]] = None,
    warnings: Optional[List[str]] = None,
    checklist: Optional[List[str]] = None,
) -> str:
    """Generate step-by-step instructions in markdown format with optional notes, warnings, and checklist."""
    
    md = f"## {title}\n"
    md += f"**Short summary:** {summary}\n\n"
    md += "### Steps\n"
    for i, step in enumerate(steps, 1):
        md += f"{i}. **{step['title']}**\n"
        for d in step["details"]:
            md += f"   - {d}\n"
        md += "\n"

    if notes:
        for n in notes:
            md += f"> **Note:** {n}\n"
    if warnings:
        for w in warnings:
            md += f"> **Warning:** {w}\n"
    if checklist:
        md += "\n**Checklist**\n"
        for c in checklist:
            md += f"✅ {c}\n"

    return md

tools = [swap_tokens, send_tokens, give_instructions]


##### LLM Setup  
This block initializes the **ChatGoogleGenerativeAI** model using parameters stored in the `config` file:  

- **model** → selects which LLM to use.  
- **temperature** → controls randomness in responses.  
- **max_output_tokens** → limits the length of generated outputs.  
- **system_instruction** → provides a system prompt to guide behavior.  

Finally, `.bind_tools(tools)` connects the LLM to our custom tools (`swap`, `send`, `give_instruction`). This allows the model to not only generate text but also take actions by invoking the tools during execution.  


In [8]:
llm = ChatGoogleGenerativeAI(
    model=config["model"],
    temperature=config["temperature"],
    max_output_tokens=config["max_output_tokens"],
    model_kwargs={"system_instruction": config["system_prompt"]}
).bind_tools(tools)

In [9]:
MAX_CONTEXT = 16

def swap_agent(state: AgentState) -> AgentState:
    # Always include system prompt at the start
    system_prompt = SystemMessage(content=DEFAULT_SYSTEM_PROMPT)

    # Keep only the last N messages
    messages = list(state["messages"])[-MAX_CONTEXT:]

    # Call LLM with system prompt + trimmed history
    response = llm.invoke([system_prompt] + messages)

    # Just return updated messages (response may be normal text or tool call)
    return {"messages": messages + [response]}


In [10]:
def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    if not last_message.tool_calls:
        return "end"
    else: 
        return "continue"

In [11]:
# Create a new graph with the defined state
graph = StateGraph(AgentState)

# Add nodes
graph.add_node("swap_agent", swap_agent)

tool_node = ToolNode(tools=tools)
graph.add_node("tools", tool_node)

# Set entry point
graph.set_entry_point("swap_agent")

# Define edges

# The llm node can go to tool or end based on the condition
graph.add_conditional_edges(
    "swap_agent",
    should_continue,
    {
        "continue": "tools",   # if tool call, go to tools
        "end": END     # else just end (LLM reply is final)
    },
)


graph.add_edge("tools", "swap_agent")

# compile the graph
app = graph.compile()

In [12]:
def chat_with_agent(user_input: str, history=None):
    if history is None:
        history = []
    
    history.append(HumanMessage(content=user_input))
    state = {"messages": history}
    result = app.invoke(state)
    history.extend([m for m in result["messages"] if isinstance(m, AIMessage)])
    
    return history


In [14]:
history = []
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    history = chat_with_agent(user_input, history)
    print("AI:", history[-1].content)


AI: I can help you swap tokens or send tokens on the Base network. Just tell me what you need! 
AI: Got it! I can help with that.

What is the amount of USDC you'd like to send, and what's the recipient's wallet address?
AI: I've sent 5 USDC to 0xcccgtksfrj. You can view the transaction details [here](https://example.com/tx/0xmocksend456).


c:\Users\Kamiye\Desktop\swap-agent\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:2007: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


AI:  


c:\Users\Kamiye\Desktop\swap-agent\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:2007: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


AI: 


c:\Users\Kamiye\Desktop\swap-agent\.venv\Lib\site-packages\langchain_google_genai\chat_models.py:2007: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


AI: I'm sorry, I cannot fulfill this request. The transaction could not be completed. Please try again.
